# A. Imports

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter

RANDOM_STATE = 42  # for reproducibility

# B. Cleaning

## Keep only first episode

In [20]:
def clean_patient_ids(INPUT_CSV, OUTPUT_CSV):
    df = pd.read_csv(INPUT_CSV)
    pid = df["PatientID"].astype(str)
    extracted = pid.str.extract(r"^(?P<base>\d+)(?:_(?P<suf>\d+))?$")

    # Determine which rows to keep:
    # - Keep if suffix is NaN (no suffix) or equals 1
    # - Drop if suffix is >= 2
    suf_num = pd.to_numeric(extracted["suf"], errors="coerce")
    keep_mask = suf_num.isna() | (suf_num == 1)
    clean = df.loc[keep_mask].copy()
    clean.loc[:, "PatientID"] = extracted.loc[keep_mask, "base"].astype(str)
    clean.to_csv(OUTPUT_CSV, index=False)

    # print("Before:")
    # print(df.head(3))
    # print("\nAfter (cleaned):")
    # print(clean.head(3))

clean_patient_ids("raw/notes.csv", "processed/notes.csv")
clean_patient_ids("raw/ehr.csv", "processed/ehr.csv")

## Keep only patients with both ehr and notes

In [21]:
NOTES_PATH = "processed/notes.csv"
EHR_PATH   = "processed/ehr.csv"
NOTES_OUT  = "processed/notes.csv"
EHR_OUT    = "processed/ehr.csv"

notes = pd.read_csv(NOTES_PATH, dtype={"PatientID": "string"})
ehr   = pd.read_csv(EHR_PATH,   dtype={"PatientID": "string"})

# normalize IDs a bit (handle stray spaces / empty)
notes["PatientID"] = notes["PatientID"].str.strip()
ehr["PatientID"]   = ehr["PatientID"].str.strip()
notes = notes.dropna(subset=["PatientID"])
ehr   = ehr.dropna(subset=["PatientID"])

# ---- find intersection ----
ids_notes = set(notes["PatientID"].unique())
ids_ehr   = set(ehr["PatientID"].unique())
ids_both  = ids_notes & ids_ehr

# ---- filter to the same set ----
notes_f = notes[notes["PatientID"].isin(ids_both)].copy()
ehr_f   = ehr[ehr["PatientID"].isin(ids_both)].copy()

# ---- (optional) sort for readability ----
notes_f = notes_f.sort_values(["PatientID"]).reset_index(drop=True)
ehr_f   = ehr_f.sort_values(["PatientID"]).reset_index(drop=True)

# ---- save ----
notes_f.to_csv(NOTES_OUT, index=False)
ehr_f.to_csv(EHR_OUT,   index=False)

# ---- report ----
print("=== BEFORE ===")
print(f"Notes: {len(notes):,} rows | {len(ids_notes):,} unique patients")
print(f"EHR  : {len(ehr):,} rows | {len(ids_ehr):,} unique patients")
print("\n=== AFTER (kept only patients present in BOTH) ===")
print(f"Common patients kept: {len(ids_both):,}")
print(f"Notes.filtered.csv: {len(notes_f):,} rows | {notes_f['PatientID'].nunique():,} patients")
print(f"ehr.filtered.csv  : {len(ehr_f):,} rows | {ehr_f['PatientID'].nunique():,} patients")

=== BEFORE ===
Notes: 204,008 rows | 31,027 unique patients
EHR  : 242,217 rows | 33,469 unique patients

=== AFTER (kept only patients present in BOTH) ===
Common patients kept: 31,027
Notes.filtered.csv: 204,008 rows | 31,027 patients
ehr.filtered.csv  : 231,100 rows | 31,027 patients


# C. Check

## Check Notes

In [22]:
# --- Load the CSV ---
df = pd.read_csv("processed/notes.csv")

# --- Functions to count ---
def char_count(text):
    return len(text)

def word_count(text):
    return len(text.split())

# --- Apply counts ---
df["char_count"] = df["Text"].astype(str).apply(char_count)
df["word_count"] = df["Text"].astype(str).apply(word_count)

# --- Overall stats ---
overall = {
    "max_chars": df["char_count"].max(),
    "avg_chars": df["char_count"].mean(),
    "max_words": df["word_count"].max(),
    "avg_words": df["word_count"].mean(),
}

print("\n🌍 Overall stats:")
print(overall)


🌍 Overall stats:
{'max_chars': 15389, 'avg_chars': np.float64(1177.5311752480295), 'max_words': 3716, 'avg_words': np.float64(224.1933600643112)}


## Check EHR

In [ ]:
df = pd.read_csv("processed/ehr.csv")
print("Shape:", df.shape)

pat_any = df.groupby("PatientID")[["Outcome","Readmission"]].max()

print("Outcome positive ratio (per patient, ANY episode):", pat_any["Outcome"].mean())
print("Readmission positive ratio (per patient, ANY episode):", pat_any["Readmission"].mean())

Shape: (231100, 64)
Outcome positive ratio (per patient, ANY episode): 0.14751622674167028
Readmission positive ratio (per patient, ANY episode): 0.19244482907832108
Outcome positive ratio (per patient, ANY episode): 0.1043284880910175
Readmission positive ratio (per patient, ANY episode): 0.13539820156637766


# D. Split

In [27]:
df = pd.read_csv('processed/ehr.csv')

pat = (
    df.groupby('PatientID', as_index=False)
      .agg(
          Outcome=('Outcome','first'),
          Readmission=('Readmission','first')
      )
)

pat['joint'] = pat['Outcome'].astype(int)*2 + pat['Readmission'].astype(int)
print("Patient counts per joint class (O*2+R):", Counter(pat['joint']))

Patient counts per joint class (O*2+R): Counter({0: 26775, 3: 3186, 1: 1015, 2: 51})


In [28]:
# 1) Hold out 20% for TEST
pat_trainval, pat_test = train_test_split(
    pat,
    test_size=0.20,
    stratify=pat['joint'],
    random_state=RANDOM_STATE
)

# 2) From the remaining 80%, carve out 12.5% as VAL  (12.5% of 80% = 10% overall)
pat_train, pat_val = train_test_split(
    pat_trainval,
    test_size=0.125,
    stratify=pat_trainval['joint'],
    random_state=RANDOM_STATE
)

len(pat_train), len(pat_val), len(pat_test)

(21718, 3103, 6206)

In [29]:
train_ids = set(pat_train['PatientID'])
val_ids   = set(pat_val['PatientID'])
test_ids  = set(pat_test['PatientID'])

train_df = df[df['PatientID'].isin(train_ids)].copy()
val_df   = df[df['PatientID'].isin(val_ids)].copy()
test_df  = df[df['PatientID'].isin(test_ids)].copy()

for name, d in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
    print(f"{name}: rows={len(d):,}, patients={d['PatientID'].nunique():,}")

Train: rows=162,298, patients=21,718
Val: rows=22,577, patients=3,103
Test: rows=46,225, patients=6,206


In [30]:
def summarize_split(name, df_rows, df_pat):
    # Row-level percentages
    o_row = df_rows['Outcome'].mean()
    r_row = df_rows['Readmission'].mean()
    # Patient-level percentages
    o_pat = df_pat['Outcome'].mean()
    r_pat = df_pat['Readmission'].mean()
    print(f"{name} — Outcome: rows={o_row:.3%}, patients={o_pat:.3%} | "
          f"Readmission: rows={r_row:.3%}, patients={r_pat:.3%}")

summarize_split("Train", train_df, pat_train)
summarize_split("Val",   val_df,   pat_val)
summarize_split("Test",  test_df,  pat_test)

# Also show joint-label distribution at patient level for each split
for name, p in [("Train", pat_train), ("Val", pat_val), ("Test", pat_test)]:
    counts = Counter(p['joint'])
    total = len(p)
    frac = {k: f"{v/total:.2%}" for k,v in counts.items()}
    print(f"{name} joint distribution (O*2+R):", frac)

Train — Outcome: rows=14.756%, patients=10.434% | Readmission: rows=19.348%, patients=13.542%
Val — Outcome: rows=16.056%, patients=10.442% | Readmission: rows=19.870%, patients=13.535%
Test — Outcome: rows=14.098%, patients=10.425% | Readmission: rows=18.577%, patients=13.535%
Train joint distribution (O*2+R): {0: '86.29%', 3: '10.27%', 1: '3.27%', 2: '0.17%'}
Val joint distribution (O*2+R): {0: '86.30%', 3: '10.28%', 1: '3.25%', 2: '0.16%'}
Test joint distribution (O*2+R): {0: '86.30%', 3: '10.26%', 1: '3.27%', 2: '0.16%'}


In [32]:
train_df.to_csv('splits/train.csv', index=False)
val_df.to_csv('splits/val.csv', index=False)
test_df.to_csv('splits/test.csv', index=False)
print("Saved splits/train.csv, splits/val.csv, splits/test.csv")

Saved splits/train.csv, splits/val.csv, splits/test.csv
